In [29]:
import pandas as pd
from pandas.io.json import json_normalize
import matplotlib.pyplot as plt
import requests
import json
import time
import numpy as np
from yelpapi import YelpAPI
from pprint import pprint
from API import api_key

In [30]:
url = 'https://api.yelp.com/v3/businesses/search'
offset = 0
API = api_key
headers = {'Authorization': 'bearer %s' % API}
business = []

for i in range(0, 500, 50):
    params = {'location': 'Cleveland, Ohio', 'offset': offset, 'limit': 50}
    response = requests.get(url, params=params, headers=headers)
    business.append(response.json())
    offset += 50
    
    if offset % 50 == 0:
        time.sleep(5)
        
business

[{'businesses': [{'id': 'J1RDyyPxhioqm8c_fi4P4Q',
    'alias': 'lola-cleveland',
    'name': 'Lola',
    'image_url': 'https://s3-media1.fl.yelpcdn.com/bphoto/XuGT-ENxIoteq2fj4OTgdg/o.jpg',
    'is_closed': False,
    'url': 'https://www.yelp.com/biz/lola-cleveland?adjust_creative=TshdU4XV3-ETjsHU3-ICtw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=TshdU4XV3-ETjsHU3-ICtw',
    'review_count': 980,
    'categories': [{'alias': 'wine_bars', 'title': 'Wine Bars'},
     {'alias': 'newamerican', 'title': 'American (New)'}],
    'rating': 4.0,
    'coordinates': {'latitude': 41.49883, 'longitude': -81.69035},
    'transactions': [],
    'price': '$$$',
    'location': {'address1': '2058 E 4th St',
     'address2': '',
     'address3': '',
     'city': 'Cleveland',
     'zip_code': '44115',
     'country': 'US',
     'state': 'OH',
     'display_address': ['2058 E 4th St', 'Cleveland, OH 44115']},
    'phone': '+12166215652',
    'display_phone': '(216) 621-5652',
    

In [3]:
pprint(business)

[{'businesses': [{'alias': 'lola-cleveland',
                  'categories': [{'alias': 'wine_bars', 'title': 'Wine Bars'},
                                 {'alias': 'newamerican',
                                  'title': 'American (New)'}],
                  'coordinates': {'latitude': 41.49883, 'longitude': -81.69035},
                  'display_phone': '(216) 621-5652',
                  'distance': 3191.079270387402,
                  'id': 'J1RDyyPxhioqm8c_fi4P4Q',
                  'image_url': 'https://s3-media1.fl.yelpcdn.com/bphoto/XuGT-ENxIoteq2fj4OTgdg/o.jpg',
                  'is_closed': False,
                  'location': {'address1': '2058 E 4th St',
                               'address2': '',
                               'address3': '',
                               'city': 'Cleveland',
                               'country': 'US',
                               'display_address': ['2058 E 4th St',
                                                   'Clevela

                  'coordinates': {'latitude': 41.4808097530181,
                                  'longitude': -81.7125004251312},
                  'display_phone': '(216) 961-5199',
                  'distance': 3443.3216298111456,
                  'id': 'OKYp_AwhXfqL_0Pgw1WB9Q',
                  'image_url': 'https://s3-media1.fl.yelpcdn.com/bphoto/RTPuoEAd_OUhg6XayoaA8g/o.jpg',
                  'is_closed': False,
                  'location': {'address1': '3900 Lorain Ave',
                               'address2': '',
                               'address3': '',
                               'city': 'Cleveland',
                               'country': 'US',
                               'display_address': ['3900 Lorain Ave',
                                                   'Cleveland, OH 44113'],
                               'state': 'OH',
                               'zip_code': '44113'},
                  'name': 'Jack Flaps',
                  'phone': '+121696

                  'display_phone': '(216) 664-0941',
                  'distance': 3144.3393611028564,
                  'id': 'fja2EnaSxKu1D7uctMhw-w',
                  'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/Ofm0FCfl-9Xu7h9x-x9dhA/o.jpg',
                  'is_closed': False,
                  'location': {'address1': '417 Prospect Ave E',
                               'address2': '',
                               'address3': '',
                               'city': 'Cleveland',
                               'country': 'US',
                               'display_address': ['417 Prospect Ave E',
                                                   'Cleveland, OH 44115'],
                               'state': 'OH',
                               'zip_code': '44115'},
                  'name': 'Red The Steakhouse',
                  'phone': '+12166640941',
                  'price': '$$$',
                  'rating': 4.0,
                  'review_count': 202,
   

                  'distance': 6908.330335304932,
                  'id': 'ZxYPXtZQ-6DFyl4VQkTELQ',
                  'image_url': 'https://s3-media3.fl.yelpcdn.com/bphoto/EDFvgUIF0oV1IVm-fss_fA/o.jpg',
                  'is_closed': False,
                  'location': {'address1': '2203 Cornell Rd',
                               'address2': '',
                               'address3': '',
                               'city': 'Cleveland',
                               'country': 'US',
                               'display_address': ['2203 Cornell Rd',
                                                   'Cleveland, OH 44106'],
                               'state': 'OH',
                               'zip_code': '44106'},
                  'name': 'Washington Place Bistro',
                  'phone': '+12167916500',
                  'price': '$$',
                  'rating': 4.0,
                  'review_count': 160,
                  'transactions': [],
                  'ur

                  'name': "Johnny's Little Bar",
                  'phone': '+12168612166',
                  'price': '$$',
                  'rating': 4.0,
                  'review_count': 153,
                  'transactions': [],
                  'url': 'https://www.yelp.com/biz/johnnys-little-bar-cleveland-2?adjust_creative=TshdU4XV3-ETjsHU3-ICtw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=TshdU4XV3-ETjsHU3-ICtw'},
                 {'alias': 'aladdins-eatery-cleveland-3',
                  'categories': [{'alias': 'mediterranean',
                                  'title': 'Mediterranean'},
                                 {'alias': 'vegetarian', 'title': 'Vegetarian'},
                                 {'alias': 'mideastern',
                                  'title': 'Middle Eastern'}],
                  'coordinates': {'latitude': 41.501310857667,
                                  'longitude': -81.592675083722},
                  'display_phone': '(21

...

                                  'title': 'American (New)'},
                                 {'alias': 'pubs', 'title': 'Pubs'},
                                 {'alias': 'sandwiches',
                                  'title': 'Sandwiches'}],
                  'coordinates': {'latitude': 41.501033,
                                  'longitude': -81.698104},
                  'display_phone': '(216) 241-4141',
                  'distance': 3745.363595329713,
                  'id': 'dOIUk3wm8LU8-FUpCKa1eg',
                  'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/9bFLP7KlC79SDxgB-QLCwQ/o.jpg',
                  'is_closed': False,
                  'location': {'address1': '1264 W 3rd St',
                               'address2': '',
                               'address3': '',
                               'city': 'Cleveland',
                               'country': 'US',
                               'display_address': ['1264 W 3rd St',
                     

                  'coordinates': {'latitude': 41.49998, 'longitude': -81.56519},
                  'display_phone': '(216) 932-3512',
                  'distance': 9402.160809473558,
                  'id': '0jHZk9RrKh0uRhTRW1XIqQ',
                  'image_url': 'https://s3-media1.fl.yelpcdn.com/bphoto/Jf06yV4ww9f4apOPfudm1w/o.jpg',
                  'is_closed': False,
                  'location': {'address1': '2199 Lee Rd',
                               'address2': '',
                               'address3': '',
                               'city': 'Cleveland Heights',
                               'country': 'US',
                               'display_address': ['2199 Lee Rd',
                                                   'Cleveland Heights, OH '
                                                   '44118'],
                               'state': 'OH',
                               'zip_code': '44118'},
                  'name': "MoMo's Kebab",
                  'pho

                  'price': '$',
                  'rating': 3.0,
                  'review_count': 85,
                  'transactions': [],
                  'url': 'https://www.yelp.com/biz/yours-truly-restaurants-cleveland?adjust_creative=TshdU4XV3-ETjsHU3-ICtw&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=TshdU4XV3-ETjsHU3-ICtw'},
                 {'alias': 'city-of-cleveland-greenhouse-cleveland',
                  'categories': [{'alias': 'landmarks',
                                  'title': 'Landmarks & Historical Buildings'}],
                  'coordinates': {'latitude': 41.5357151627541,
                                  'longitude': -81.6277085244656},
                  'display_phone': '(216) 664-3103',
                  'distance': 7948.322239177737,
                  'id': 'VdvEnuQt90XYfy_1khVZAw',
                  'image_url': 'https://s3-media1.fl.yelpcdn.com/bphoto/1zlsz599KLlG81cw0C9dtQ/o.jpg',
                  'is_closed': False,
         

                  'categories': [{'alias': 'parks', 'title': 'Parks'}],
                  'coordinates': {'latitude': 41.49309, 'longitude': -81.79687},
                  'display_phone': '',
                  'distance': 10602.027814456233,
                  'id': '1HWbtB8XLQeiD6Ossp96dA',
                  'image_url': 'https://s3-media3.fl.yelpcdn.com/bphoto/6b_TJ01Zn7Eiv-iirsqY0w/o.jpg',
                  'is_closed': False,
                  'location': {'address1': 'Belle Ave & Lake Ave',
                               'address2': '',
                               'address3': '',
                               'city': 'Lakewood',
                               'country': 'US',
                               'display_address': ['Belle Ave & Lake Ave',
                                                   'Lakewood, OH 44107'],
                               'state': 'OH',
                               'zip_code': '44107'},
                  'name': 'Lakewood Park',
                

                  'categories': [{'alias': 'delis', 'title': 'Delis'},
                                 {'alias': 'bakeries', 'title': 'Bakeries'}],
                  'coordinates': {'latitude': 41.40896, 'longitude': -81.69289},
                  'display_phone': '(216) 351-1161',
                  'distance': 7368.856193287775,
                  'id': '7sqyF1oNXKWGMU0A_tVaIg',
                  'image_url': 'https://s3-media3.fl.yelpcdn.com/bphoto/1kFXwaXUSvYaOOI7q19PGw/o.jpg',
                  'is_closed': False,
                  'location': {'address1': '5626 Broadview Rd',
                               'address2': '',
                               'address3': '',
                               'city': 'Parma',
                               'country': 'US',
                               'display_address': ['5626 Broadview Rd',
                                                   'Parma, OH 44134'],
                               'state': 'OH',
                               'zi

In [28]:
response = business[0]['businesses']
df = pd.DataFrame.from_dict(response)
df

,alias,categories,coordinates,display_phone,distance,id,image_url,is_closed,location,name,phone,price,rating,review_count,transactions,url
0,lola-cleveland,"[{'alias': 'wine_bars', 'title': 'Wine Bars'},...","{'latitude': 41.49883, 'longitude': -81.69035}",(216) 621-5652,3191.079270,J1RDyyPxhioqm8c_fi4P4Q,https://s3-media1.fl.yelpcdn.com/bphoto/XuGT-E...,False,"{'address1': '2058 E 4th St', 'address2': '', ...",Lola,+12166215652,$$$,4.0,980,[],https://www.yelp.com/biz/lola-cleveland?adjust...
1,townhall-cleveland-2,"[{'alias': 'newamerican', 'title': 'American (...","{'latitude': 41.485492, 'longitude': -81.703914}",(216) 344-9400,2951.385340,LNsZJP6jZ11e0tDljOLPiQ,https://s3-media4.fl.yelpcdn.com/bphoto/ycqFgv...,False,"{'address1': '1909 W 25th St', 'address2': '',...",Townhall,+12163449400,$$,4.0,991,[],https://www.yelp.com/biz/townhall-cleveland-2?...
2,l-albatros-cleveland,"[{'alias': 'french', 'title': 'French'}]","{'latitude': 41.5117439550152, 'longitude': -8...",(216) 791-7880,6953.412053,SzcCJJmreMlWCsNjupdVAA,https://s3-media4.fl.yelpcdn.com/bphoto/QpVBtp...,False,"{'address1': '11401 Bellflower Rd', 'address2'...",L'Albatros,+12167917880,$$$,4.5,603,[],https://www.yelp.com/biz/l-albatros-cleveland?...
3,slymans-restaurant-cleveland,"[{'alias': 'tradamerican', 'title': 'American ...","{'latitude': 41.51286, 'longitude': -81.67125}",(216) 621-3760,4387.852814,1veVZUawy7IhIc5oDpRRQA,https://s3-media2.fl.yelpcdn.com/bphoto/rz2ST-...,False,"{'address1': '3106 Saint Clair Ave NE', 'addre...",Slyman's Restaurant,+12166213760,$$,4.5,417,[],https://www.yelp.com/biz/slymans-restaurant-cl...
4,great-lakes-brewing-company-cleveland-4,"[{'alias': 'breweries', 'title': 'Breweries'},...","{'latitude': 41.4844102, 'longitude': -81.7043...",(216) 771-4404,2944.089481,meXjqyhTNLFmknY39y2sMg,https://s3-media1.fl.yelpcdn.com/bphoto/RHNLaz...,False,"{'address1': '2516 Market Ave', 'address2': ''...",Great Lakes Brewing Company,+12167714404,$$,4.0,850,[],https://www.yelp.com/biz/great-lakes-brewing-c...
5,sokolowskis-university-inn-cleveland,"[{'alias': 'polish', 'title': 'Polish'}, {'ali...","{'latitude': 41.48475, 'longitude': -81.69005}",(216) 771-9236,1942.420138,Xny0n0s98TpP82sQxfgIMQ,https://s3-media2.fl.yelpcdn.com/bphoto/F1jEYU...,False,"{'address1': '1201 University Rd', 'address2':...",Sokolowski's University Inn,+12167719236,$$,4.5,463,[],https://www.yelp.com/biz/sokolowskis-universit...
6,luckys-cafe-cleveland-3,"[{'alias': 'coffee', 'title': 'Coffee & Tea'},...","{'latitude': 41.47736, 'longitude': -81.68308}",(216) 622-7773,992.894340,_5PJ4GHIXNdUdXtohylKGQ,https://s3-media4.fl.yelpcdn.com/bphoto/-AYXJl...,False,"{'address1': '777 Starkweather Ave', 'address2...",Lucky's Cafe,+12166227773,$$,4.0,772,[],https://www.yelp.com/biz/luckys-cafe-cleveland...
7,momocho-cleveland-2,"[{'alias': 'mexican', 'title': 'Mexican'}]","{'latitude': 41.4840151, 'longitude': -81.7098...",(216) 694-2122,3334.932184,F2pfjAZ_3dMTGCKv6c5wOw,https://s3-media3.fl.yelpcdn.com/bphoto/D61sVw...,False,"{'address1': '1835 Fulton Rd', 'address2': Non...",Momocho,+12166942122,$$,4.0,694,[],https://www.yelp.com/biz/momocho-cleveland-2?a...
8,happy-dog-cleveland,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...","{'latitude': 41.4848327636719, 'longitude': -8...",(216) 651-9474,4685.922534,anzCdD5eIEHwydTpXhLlUA,https://s3-media2.fl.yelpcdn.com/bphoto/2r3m49...,False,"{'address1': '5801 Detroit Ave', 'address2': '...",Happy Dog,+12166519474,$,4.0,605,[],https://www.yelp.com/biz/happy-dog-cleveland?a...
9,superior-pho-cleveland,"[{'alias': 'vietnamese', 'title': 'Vietnamese'...","{'latitude': 41.51056, 'longitude': -81.6698}",(216) 781-7462,4127.849575,AbPQf-X7awuPFDULiJ43bg,https://s3-media2.fl.yelpcdn.com/bphoto/Vb88rq...,False,"{'address1': '3030 Superior Ave E', 'address2'...",Superior Pho,+12167817462,$,4.0,580,[pickup],https://www.yelp.com/biz/superior-pho-clevelan...


In [26]:
data = pd.DataFrame()
l=[] 

for dataFrame in business:
    print(type(pd.DataFrame.from_dict(dataFrame['businesses'])))
    l.append(pd.DataFrame.from_dict(dataFrame['businesses']))
data=pd.concat(l)
data.insert(0,'index',range(len(data)))


<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>


In [27]:
data

,index,alias,categories,coordinates,display_phone,distance,id,image_url,is_closed,location,name,phone,price,rating,review_count,transactions,url
0,0,lola-cleveland,"[{'alias': 'wine_bars', 'title': 'Wine Bars'},...","{'latitude': 41.49883, 'longitude': -81.69035}",(216) 621-5652,3191.079270,J1RDyyPxhioqm8c_fi4P4Q,https://s3-media1.fl.yelpcdn.com/bphoto/XuGT-E...,False,"{'address1': '2058 E 4th St', 'address2': '', ...",Lola,+12166215652,$$$,4.0,980,[],https://www.yelp.com/biz/lola-cleveland?adjust...
1,1,townhall-cleveland-2,"[{'alias': 'newamerican', 'title': 'American (...","{'latitude': 41.485492, 'longitude': -81.703914}",(216) 344-9400,2951.385340,LNsZJP6jZ11e0tDljOLPiQ,https://s3-media4.fl.yelpcdn.com/bphoto/ycqFgv...,False,"{'address1': '1909 W 25th St', 'address2': '',...",Townhall,+12163449400,$$,4.0,991,[],https://www.yelp.com/biz/townhall-cleveland-2?...
2,2,l-albatros-cleveland,"[{'alias': 'french', 'title': 'French'}]","{'latitude': 41.5117439550152, 'longitude': -8...",(216) 791-7880,6953.412053,SzcCJJmreMlWCsNjupdVAA,https://s3-media4.fl.yelpcdn.com/bphoto/QpVBtp...,False,"{'address1': '11401 Bellflower Rd', 'address2'...",L'Albatros,+12167917880,$$$,4.5,603,[],https://www.yelp.com/biz/l-albatros-cleveland?...
3,3,slymans-restaurant-cleveland,"[{'alias': 'tradamerican', 'title': 'American ...","{'latitude': 41.51286, 'longitude': -81.67125}",(216) 621-3760,4387.852814,1veVZUawy7IhIc5oDpRRQA,https://s3-media2.fl.yelpcdn.com/bphoto/rz2ST-...,False,"{'address1': '3106 Saint Clair Ave NE', 'addre...",Slyman's Restaurant,+12166213760,$$,4.5,417,[],https://www.yelp.com/biz/slymans-restaurant-cl...
4,4,great-lakes-brewing-company-cleveland-4,"[{'alias': 'breweries', 'title': 'Breweries'},...","{'latitude': 41.4844102, 'longitude': -81.7043...",(216) 771-4404,2944.089481,meXjqyhTNLFmknY39y2sMg,https://s3-media1.fl.yelpcdn.com/bphoto/RHNLaz...,False,"{'address1': '2516 Market Ave', 'address2': ''...",Great Lakes Brewing Company,+12167714404,$$,4.0,850,[],https://www.yelp.com/biz/great-lakes-brewing-c...
5,5,sokolowskis-university-inn-cleveland,"[{'alias': 'polish', 'title': 'Polish'}, {'ali...","{'latitude': 41.48475, 'longitude': -81.69005}",(216) 771-9236,1942.420138,Xny0n0s98TpP82sQxfgIMQ,https://s3-media2.fl.yelpcdn.com/bphoto/F1jEYU...,False,"{'address1': '1201 University Rd', 'address2':...",Sokolowski's University Inn,+12167719236,$$,4.5,463,[],https://www.yelp.com/biz/sokolowskis-universit...
6,6,luckys-cafe-cleveland-3,"[{'alias': 'coffee', 'title': 'Coffee & Tea'},...","{'latitude': 41.47736, 'longitude': -81.68308}",(216) 622-7773,992.894340,_5PJ4GHIXNdUdXtohylKGQ,https://s3-media4.fl.yelpcdn.com/bphoto/-AYXJl...,False,"{'address1': '777 Starkweather Ave', 'address2...",Lucky's Cafe,+12166227773,$$,4.0,772,[],https://www.yelp.com/biz/luckys-cafe-cleveland...
7,7,momocho-cleveland-2,"[{'alias': 'mexican', 'title': 'Mexican'}]","{'latitude': 41.4840151, 'longitude': -81.7098...",(216) 694-2122,3334.932184,F2pfjAZ_3dMTGCKv6c5wOw,https://s3-media3.fl.yelpcdn.com/bphoto/D61sVw...,False,"{'address1': '1835 Fulton Rd', 'address2': Non...",Momocho,+12166942122,$$,4.0,694,[],https://www.yelp.com/biz/momocho-cleveland-2?a...
8,8,happy-dog-cleveland,"[{'alias': 'bars', 'title': 'Bars'}, {'alias':...","{'latitude': 41.4848327636719, 'longitude': -8...",(216) 651-9474,4685.922534,anzCdD5eIEHwydTpXhLlUA,https://s3-media2.fl.yelpcdn.com/bphoto/2r3m49...,False,"{'address1': '5801 Detroit Ave', 'address2': '...",Happy Dog,+12166519474,$,4.0,605,[],https://www.yelp.com/biz/happy-dog-cleveland?a...
9,9,superior-pho-cleveland,"[{'alias': 'vietnamese', 'title': 'Vietnamese'...","{'latitude': 41.51056, 'longitude': -81.6698}",(216) 781-7462,4127.849575,AbPQf-X7awuPFDULiJ43bg,https://s3-media2.fl.yelpcdn.com/bphoto/Vb88rq...,False,"{'address1': '3030 Superior Ave E', 'address2'...",Superior Pho,+12167817462,$,4.0,580,[pickup],https://www.yelp.com/biz/superior-pho-clevelan...
